In [4]:
library(tidyverse)
library(tsibble)
library(fable)

### Import the housing data

In [5]:
df <- read.csv('monthly_housing_inventory.csv')
df %>% dim()

[1] 288385     10

### Extract county, state, and create date variable
The `county_name` contains the `county` and `state`. Use `regex` to extract those into separate fields. Convert the `month_date_yyyymm` from an integer to a `monthly date`.


In [7]:
df <-  df %>% 
mutate(county = str_extract(county_name, '^(.*)?, ([a-z]{2})$', group=1)) %>%
mutate(state = str_extract(county_name, '^(.*)?, ([a-z]{2})$', group=2)) %>%
mutate(month_date = tsibble::yearmonth(as.character(month_date_yyyymm), '%Y%m'))

df %>% head()

,month_date_yyyymm,county_name,median_listing_price,active_listing_count,new_listing_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,county,state,month_date
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<mth>
1,202403,"hartley, tx",425000,12,0,1,289,3024,444336,12,hartley,tx,2024 Mar
2,202403,"rock, mn",309900,21,8,13,136,2016,397205,35,rock,mn,2024 Mar
3,202403,"carter, ok",199000,165,56,59,127,1633,306719,222,carter,ok,2024 Mar
4,202403,"mckean, pa",169000,48,12,25,89,1898,187270,71,mckean,pa,2024 Mar
5,202403,"etowah, al",248950,236,128,67,124,1894,296370,303,etowah,al,2024 Mar
6,202403,"dawson, ga",679450,135,76,80,227,2802,814852,211,dawson,ga,2024 Mar


In [8]:
# If doing option 3 you may want to filter the data ahead of time to reduce the processing
df_five_states <- df %>% filter(state %in% c('md', 'va', 'pa', 'de', 'nj'))
df_five_states %>% dim()

[1] 22674    13

In [10]:
df_ts <- df %>% as_tsibble(index=month_date, key=c(state, county))
df_five_states_ts <- df_five_states %>% as_tsibble(index=month_date, key=c(state, county))

In [11]:
df_five_states_ts <- df_five_states_ts %>%
fill_gaps()

df_five_states_ts %>% head()

month_date_yyyymm,county_name,median_listing_price,active_listing_count,new_listing_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,county,state,month_date
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<mth>
201607,"kent, de",240000,1203,332,412,119,1916,248196,1596,kent,de,2016 Jul
201608,"kent, de",238925,1202,282,389,118,1911,244720,1588,kent,de,2016 Aug
201609,"kent, de",239000,1192,362,278,119,1898,245363,1461,kent,de,2016 Sep
201610,"kent, de",239700,1184,272,410,119,1909,247105,1603,kent,de,2016 Oct
201611,"kent, de",238600,1160,274,382,119,1909,245420,1544,kent,de,2016 Nov
201612,"kent, de",235780,1099,184,346,118,1897,241322,1443,kent,de,2016 Dec


In [12]:
df_five_states_ts %>% dim()

[1] 22802    13

### Sample Filling for the Five States in Option-3

In [15]:
df_five_states_ts_filled <- df_five_states_ts |>
  model(ARIMA(active_listing_count ~ pdq(0:1, 0:1, 0:1) + PDQ(0,0,0))) |>
  fabletools::interpolate(df_five_states_ts)

df_five_states_ts_filled %>% filter(is.na(active_listing_count)) %>% dim()

[1] 0 4

In [17]:
df_five_states %>% count(state, county)

state,county,n
<chr>,<chr>,<int>
de,kent,93
de,new castle,93
de,sussex,93
md,allegany,93
md,anne arundel,93
md,baltimore,93
md,baltimore city,93
md,calvert,93
md,caroline,93


### Sample Aggregation for National Data
Probably want to do this before you set as a tsibble, otherwise the aggregation gets problematic due to the inner workings of the tsibble package.

In [ ]:
df %>%
mutate(feet = median_square_feet * total_listing_count) %>%
group_by(month_date_yyyymm) %>%
summarise(
    total_listings = sum(total_listing_count, na.rm = TRUE),
    feet = sum(feet, na.rm=TRUE)
, .groups='drop') %>%
mutate(wgt_median_square_feet = feet / total_listings) %>%
mutate(month_date = tsibble::yearmonth(as.character(month_date_yyyymm), '%Y%m')) %>%
select(-feet, -month_date_yyyymm) %>%
as_tsibble(index=month_date) %>%
head()

total_listings,wgt_median_square_feet,month_date
<dbl>,<dbl>,<mth>
1873362,1984.003,2016 Jul
1845212,1974.786,2016 Aug
1809143,1970.087,2016 Sep
1753435,1964.404,2016 Oct
1664952,1956.831,2016 Nov
1522590,1941.071,2016 Dec


### Sample Aggregate Set-up
For Simplicity, I would only pick a metric that can be summed.

In [ ]:
df_ts %>%
fabletools::aggregate_key(
    state / county,
    total_listing=sum(total_listing_count),
    pending_listings=sum(pending_listing_count,na.rm=TRUE)
) %>%
head()

month_date,state,county,total_listing,pending_listings
<mth>,<chr*>,<chr*>,<dbl>,<dbl>
2016 Jul,<aggregated>,<aggregated>,1873362,414444
2016 Aug,<aggregated>,<aggregated>,1845212,393440
2016 Sep,<aggregated>,<aggregated>,1809143,370710
2016 Oct,<aggregated>,<aggregated>,1753435,352650
2016 Nov,<aggregated>,<aggregated>,1664952,331782
2016 Dec,<aggregated>,<aggregated>,1522590,292734
